<img src="https://storage.googleapis.com/dm-educational/assets/ai_foundations/GDM-Labs-banner-image-C6-white-bg.png">

# Lab: Estimate Training FLOPs

<a href='https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_7/gdm_lab_7_2_estimate_training_flops.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

Learn how to estimate compute costs using FLOPs.

15 minutes

##Overview
So far, you have observed the computations involved in training language models and how computationally expensive these operations can be. You have also learned about the important role GPUs play by performing matrix multiplication operations in parallel to speed up the training process.

Since it can be very computationally intensive to train models, especially as the model size increases, it is important to estimate the compute required before you begin training. This estimate is important for three reasons:

* To estimate the monetary cost of training.
* To understand whether it is possible to train a particular model on given hardware.
* To estimate how long training the model might take.

In this lab, you will learn how to estimate the computational cost of training a language model. The primary metric we use to measure this cost is **FLOPs**, which stands for **FL**oating-point **OP**erations.

### What you will learn:

By the end of this lab, you will be able to:

* Describe FLOPs and explain why they are used as a measure of computation.
* Compute a simple formula to estimate the FLOPs required for training.

###Tasks
**In this lab, you will:**
* Write a function to compute the number of FLOPs.
* Explore how the size of the model and dataset chosen impacts the number of FLOPs required for training.


## How to use Google Colaboratory (Colab)

Google Colaboratory (also known as Google Colab) is a platform that allows you to run Python code in your browser. The code is written in **cells** that are excuted on a remote server.

To run a cell, hover over a cell and click on the `run` button to its left. The run button is the circle with the triangle (▶). Alternatively, you can also click on a cell and use the keyboard combination Ctrl+Return (or ⌘+Return if you are using a Mac).

To try this out, run the following cell. This should print today's day of the week below it.

In [ ]:
from datetime import datetime
print(f"Today is {datetime.today():%A}.")

Note that the **order in which you run the cells matters**. When you are working through a lab, make sure to always run all cells in order, otherwise the code might not work. If you take a break while working on a lab, Colab may disconnect you and in that case, you have to execute all cells again before  continuing your work. To make this easier, you can select the cell you are currently working on and then choose _Runtime → Run before_  from the menu above (or use the keyboard combination Ctrl/⌘ + F8). This will re-execute all cells before the current one.

## Imports

In this lab, you will make use of the [`pandas`](http://pandas.pydata.org) package for displaying results in a table. Run the following cell to import this package.



In [ ]:
%%capture
# Install the custom package for this course.
!pip install "git+https://github.com/google-deepmind/ai-foundations.git@main"

# Packages used.
import pandas as pd # For loading the dataset.
from ai_foundations.utils import formatting # For formatting large numbers.
from ai_foundations.feedback.course_7 import flops # For providing feedback.

# For formatting the model generations.
from IPython.display import display, HTML

## What are FLOPs?

Recall that each matrix multiplication in a transformer block is composed of multiplication and addition operations. Each operation is a **floating point operation**, or **FLOP**, since it involves calculations between floating point numbers.

To measure the total computational cost of a task, such as training a model, you can count up all of these individual operations. The total count is referred to as the number of **FLOPs**. In the example provided in the previous article "What happens in the GPU during training and during inference?", a simple matrix multiplication between a $2 \times 3$ matrix and a $3 \times 2$ matrix required 20 operations. However, training a language model can require billions, trillions or even more of these operations.

------
> **ℹ️ Info: FLOPs versus FLOPS:**
>
>You will often see two very similar acronyms: FLOPs and FLOPS. The capitalization of the final S is crucial as it signals a different meaning.
>
>1. **FLOPs** (Floating Point Operations) is the plural of **FLOP**. It represents the total number of operations required to perform a specific training process or perform inference on a specific example.
>
>   * For example: "Training this model will require 500 trillion FLOPs."
>
>2. **FLOPS** (Floating Point Operations Per Second) is a rate of measurement. It is a measure of computational performance, or speed. It tells you how many floating point operations (FLOPs) a piece of specific hardware (e.g., a GPU) can perform per second. The capitalized S therefore stands for "per Second."
>
>   * For example: "This GPU has a performance of 200 TeraFLOPS ($200 \times 10^{12}$ FLOPS)."
>
>To see the connection between FLOPs and FLOPS, consider a model training process that requires 1,000 FLOPs on a GPU that processes at 100 FLOPS. In this case, it would take 10 seconds to train the model, since the GPU can perform 100 FLOPs per second (i.e., 100 FLOPS).
>
------

## Estimating FLOPs

A widely used "back-of-the-envelope" formula for estimating the number of FLOPs required for one epoch of training (i.e., one full pass over the dataset) is:

$$\text{FLOPs} \approx 6 \times P \times N$$


where $P$ is the total number of trainable model parameters and $N$ is the total number of tokens in the training dataset.

The scaling factor of 6 is a widely used rule of thumb that accounts for the work done during one training step:

* **Forward pass**: The cost is approximately $2 \times \text{P} \times \text{N}$ FLOPs. The factor of 2 comes from the fact that, for each parameter, there is roughly one multiplication and one addition (a "multiply-accumulate" operation).

* **Backward pass**: The cost is approximately $4 \times \text{P} \times \text{N}$ FLOPs. This is because the backward pass, which involves calculating gradients, is roughly twice as computationally expensive as the forward pass [1].

Adding these two components together gives the final estimation of $6 \times P \times N$.

You can read more about methods for estimating FLOPs in the paper [Training Compute-Optimal Large Language Models](https://arxiv.org/abs/2203.15556) from Google DeepMind [2].


## Coding Activity 1: Implement the FLOPs estimation formula

In preparation for computing FLOPs, you need a function that returns the number of FLOPs.

------
> 💻 **Your task**:
>
> Complete the function `compute_num_flops(param_count: float, num_tokens: float)`.
>
> It should return a single number representing the number of FLOPs according to the estimation formula.
>
> Once you have implemented this function, run both cells to define the function and test your code.
------

In [ ]:
def compute_num_flops(param_count: float, num_tokens: float) -> float:
    """Estimates the training FLOPs for one epoch.

    Args:
      param_count: The total number of trainable parameters in the model.
      num_tokens: The total number of tokens in the training dataset.

    Returns:
      The estimated total number of floating point operations.
    """
    total_flops = ... # Add your code here.

    return total_flops

In [ ]:
# @title Run this cell to test your implementation.
flops.test_compute_num_flops(compute_num_flops)

## Coding Activity 2: Estimate training costs

Your next task is to compute the estimated training FLOPs for models and datasets of different sizes. The example scenarios given are all real-world models and datasets developed by Google.

------
> 💻 **Your task**:
>
>1. Use the first cell to calculate the estimated FLOPs for each of the following examples using your implemented `compute_num_flops` function.
>
>2. Add these FLOPs values to the dictionary in the second cell.
>
>**Examples**
>
>BERT-base [3]
>- Model parameters: 110 million
>- Dataset tokens: 3.3 billion
>
>T5-large [4]
>- Model parameters: 770 million
>- Dataset tokens: 1 trillion
>
>Gemma-1B and Africa Galore dataset
>- Model parameters: 1 billion
>- Dataset tokens: 30 thousand
>
>PaLM [5]
>- Model parameters: 540 billion
>- Dataset tokens: 780 billion
>
------

<br>

------
> **ℹ️ Info: Scientific notation in Python**
>
> To make it easier to write large numbers like billions and trillions in the code, you can use Python's scientific notation with the letter `e`. The `e` simply means "times 10 to the power of."

>For example:
>
>- 1 million (1,000,000) can be written as `1e6` (a 1 with 6 zeros).
>
>- 2.5 billion (2,500,000,000) can be written as `2.5e9`.
>
>- 15 trillion (15,000,000,000,000) can be written as `1.5e13`.
>
>Using this notation will also help you avoid typos when entering the large numbers from the table into the dictionary.
------

In [ ]:
# @markdown Enter the values for a scenario to calculate the FLOPs.

param_count = 1e9  # @param {type:"number"}
num_tokens = 30e3  # @param {type:"number"}

# Calculate and display the result using your function.
total_flops = compute_num_flops(param_count, num_tokens)

output_string = f"""
  <p>
      For a model with <b>{formatting.format_large_number(param_count)}
      parameters</b>
      trained on <b>{formatting.format_large_number(num_tokens)} tokens</b>:
  </p>
  <hr>
  <p><b>Estimated training FLOPs:</b> {formatting.format_flops(total_flops)}</p>
"""
display(HTML(output_string))

Fill in the following cell with your calculated FLOPs values.

In [ ]:
# Fill in all the '...' values based on your calculations.
final_results_data = [
    {
        "Scenario": "BERT-Base",
        "Parameters": formatting.format_large_number(110e6),
        "Tokens": formatting.format_large_number(3.3e9),
        "FLOPs": ...,
    },
    {
        "Scenario": "T5-Large",
        "Parameters": formatting.format_large_number(770e6),
        "Tokens": formatting.format_large_number(1e12),
        "FLOPs": ...,
    },
    {
        "Scenario": "Gemma-1B (Africa Galore)",
        "Parameters": formatting.format_large_number(1e9),
        "Tokens": formatting.format_large_number(30e3),
        "FLOPs": ...,
    },
    {
        "Scenario": "PaLM",
        "Parameters": formatting.format_large_number(540e9),
        "Tokens": formatting.format_large_number(780e9),
        "FLOPs": ...,
    },
]

# Test if results are correct.
if flops.test_final_results(final_results_data):

    for scenario in final_results_data:
        scenario["FLOPs"] = formatting.format_flops(scenario["FLOPs"])

    # Create and display the table.
    df = pd.DataFrame(final_results_data)
    display(df)

### What did you observe?

Look at the results in the table. Notice the immense jump in computational cost as models and datasets grow. Training the Gemma-1B model on the Africa Galore dataset requires a tiny fraction of the compute needed for training a much larger model like PaLM.

This exercise highlights the direct relationship between model scale and the massive hardware resources required for training.

## Coding Activity 3: Interactive exploration

------
> 💻 **Your task**:
>
> Use the interactive sliders in the following cell to further explore how changing the model and dataset size impacts the computational cost.
>
>Because these numbers grow so quickly, the sliders adjust the exponent of the number (a logarithmic scale). This means that each step on the slider increases the underlying value by a factor of 10, allowing you to move from millions, to billions, to trillions of parameters/tokens.
>
>For example, for the `log_param_count` slider:
>
>A slider value of 6 means $10^{6}$, which is 1 million parameters.
>
>A slider value of 9 means $10^{9}$, which is 1 billion parameters.
>
>A slider value of 12 means $10^{12}$, which is 1 trillion parameters.
-----

In [ ]:
# @title Estimate computational cost
# @markdown Drag the sliders to see how FLOPs scale.

log_param_count = 9  # @param {type:"slider", min:6, max:12, step:1}
log_num_tokens = 14  # @param {type:"slider", min:9, max:14, step:1}

param_count = 10**log_param_count
num_tokens = 10**log_num_tokens

# Calculate and display the result.
total_flops = compute_num_flops(param_count, num_tokens)

output_string = f"""
    <p>
        <b>Model with:</b> {formatting.format_large_number(param_count)} parameters<br>
        <b>Dataset with:</b> {formatting.format_large_number(num_tokens)} tokens
    </p>
    <hr>
    <p><b>Estimated training FLOPs:</b> {formatting.format_flops(total_flops)}</p>
"""

display(HTML(output_string))

Notice that, if you increase either the `log_param_count` or the `log_num_tokens` slider by 1, the exponent in the final "Estimated Training FLOPs" also increases by approximately 1. As this is a logarithmic scale, a change of 1 in the exponent means a 10x increase in the actual number.

This reveals a direct, proportional effect: if you increase the number of parameters by 10 times, the total computation required also increases by 10 times. This predictable, one-to-one scaling is known as a **linear relationship**. This is why training models at the trillion-parameter scale requires such an astronomical amount of computation compared to smaller models.

## Summary

This lab provided you with a practical guide for estimating the computational cost of training a language model. You encountered the concept of **floating point operations (FLOPs)** and learned that estimating the number of FLOPs is an important step to forecast monetary costs, training times, and hardware requirements for large models. You learned a method to estimate the number of FLOPs required to train a model for one epoch and you investigated the relationship between the number of parameters and the number of training tokens.

In the next activity, you will look at the energy costs associated with training models.

## Solutions

The following cells provide reference solutions to the coding activities in this notebook. If you really get stuck after trying to solve the activities yourself, you may want to consult these solutions.

It is recommended that you *only* look at the solutions after you have tried to solve the activities *multiple times*. The best way to learn challenging concepts in computer science and artificial intelligence is to debug your code piece-by-piece until it works, rather than copying existing solutions.

If you feel stuck, you may want to first try to debug your code. For example, by adding additional print statements to see what your code is doing at every step. This will provide you with a much deeper understanding of the code and the materials. It will also provide you with practice on how to solve challenging coding problems beyond this course.

To view the solutions for an activity, click on the arrow to the left of the activity name. If you consult the solutions, do not copy and paste them into the cells above. Instead, look at them, and type them manually into the cell. This will help you understand where you went wrong.

### Coding Activity 1

In [ ]:
# This is a complete implementation of `compute_num_flops`.
def compute_num_flops(param_count, num_tokens) -> float:
    """Estimates the training FLOPs for one epoch.

    Args:
      param_count: The total number of trainable parameters in the model.
      num_tokens: The total number of tokens in the training dataset.

    Returns:
      The estimated total number of floating point operations.
    """

    total_flops = 6 * param_count * num_tokens

    return total_flops

### Coding Activity 2

In [ ]:
final_results_data = [
    {
        "Scenario": "BERT-Base",
        "Parameters": formatting.format_large_number(110e6),
        "Tokens": formatting.format_large_number(3.3e9),
        "FLOPs": 2.18e18,
    },
    {
        "Scenario": "T5-Large",
        "Parameters": formatting.format_large_number(770e6),
        "Tokens": formatting.format_large_number(1e12),
        "FLOPs": 4.62e21,
    },
    {
        "Scenario": "Gemma-1B (Africa Galore)",
        "Parameters": formatting.format_large_number(1e9),
        "Tokens": formatting.format_large_number(30e3),
        "FLOPs": 1.80e14,
    },
    {
        "Scenario": "PaLM",
        "Parameters": formatting.format_large_number(540e9),
        "Tokens": formatting.format_large_number(780e9),
        "FLOPs": 2.53e24,
    },
]

## References

[1] Dzmitry Bahdanau. 2022. The FLOPs Calculus of Language Model Training. *Medium*. Retrieved from https://medium.com/@dzmitrybahdanau/the-flops-calculus-of-language-model-training-3b19c1f025e4.

[2] Jordan Hoffmann et al. 2022. [Training Compute-Optimal Large Language Models](https://arxiv.org/abs/2203.15556). arXiv preprint arXiv:2203.15556.

[3] Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova. 2018. [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805). arXiv preprint arXiv:1810.04805.

[4] Colin Raffel et al. 2019. [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/abs/1910.10683). arXiv preprint arXiv:1910.10683

[5] Aakanksha Chowdhery et al. 2022. [PaLM: Scaling Language Modeling with Pathways
](https://arxiv.org/abs/2204.02311). arXiv preprint arXiv:2204.02311.